In [2]:
import requests
import json
import html
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import re

C:\Users\Karina\miniconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [3]:
# Function to remove special characters
def normalize_name(name):
    # Normalize to remove accents and special characters
    name = unicodedata.normalize('NFD', name)
    # Encode to ASCII and ignore errors, then decode back to string
    name = name.encode('ascii', 'ignore').decode('utf-8')
    return name

In [4]:

# # URL of the JSON file
# url = "https://itra.run/Races/RaceResults/TOR330.Tor.des.G%C3%A9ants%C2%AE/2023/80226"

# # Send a GET request to fetch the JSON data
# response = requests.get(url)
# response

In [5]:
years = [ 
        '2024', # NO DNFs! AAAAAAAAAAAAAAAAAAh!
        '2023', # NO DNFs! AAAAAAAAAAAAAAAAAAh!
        '2022', 
        '2021',  # NO DNFs! AAAAAAAAAAAAAAAAAAh!
        '2020',
        '2019',
        '2018', 
#          '2017'
]

races = ['SwissPeaks360', 'SwissPeaks360_DUO']

#### Loading ITRA 

In [6]:

SP_itra_df = {}
# races = ['TOR450']

for race in races:
    for year in years:
        try:

            file = f'Data/ITRA/TXT/{race}_{year}.txt'
            df = pd.read_csv(file, delimiter='\t', 
                         dtype={'Time': 'string'})    

            # renaming columns so it is easier later to compare
            df = df.rename(columns={"Runner": "Name",
                                    "Gender": "Sex",
                                    "Nationality": "ITRA_Nationality",
                                   })

            # change the name to a title case
            df['Name'] = df['Name'].str.title()
            df['Name'] = df['Name'].apply(normalize_name)
            

            # Convert 'Time' to timedelta format
            df['Performance'] = pd.to_timedelta(df['Time'], errors='coerce')
            # Calculate total seconds and create 'Performance_Seconds' column
            df['Performance_Seconds'] = df['Performance'].dt.total_seconds()


            df['Status'] = np.where(df['Performance'].isna(), 'DNF', 'Finished')
            df['Year'] = f'{year}'
            df['Race'] = f'{race}'

    #             # Drop the intermediate columns if they're no longer needed
            df = df.drop(columns=['Time', '#'])

            df['Name'] = df['Name'].str.strip()
            df['Name'] = df['Name'].str.replace(',', '')
            df['Name'] = df['Name'].str.replace('\'', '')
            df['Name'] = df['Name'].str.replace('-', '')
            df['Name'] = df['Name'].str.replace(r"\s+", " ")

    #             print(race, year, '\n', df.head())

            # Store the DataFrame in the dictionary with a key like 'TOR330_2021'
            SP_itra_df[f'{race}_{year}'] = df
            print('* ',f'{race}_{year} {df.shape}', '\n')
        except:

            if year == '2018' :
                pass
            elif year == '2019':
                pass
            else:
                print(f'issues with {race}_{year}')
                print(f'{file}')
                print('\n')
            


C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a fu

*  SwissPeaks360_2024 (133, 9) 

*  SwissPeaks360_2023 (147, 9) 

*  SwissPeaks360_2022 (228, 9) 

*  SwissPeaks360_2021 (182, 9) 

*  SwissPeaks360_2020 (218, 9) 

*  SwissPeaks360_2019 (240, 9) 

*  SwissPeaks360_2018 (300, 9) 

*  SwissPeaks360_DUO_2024 (18, 9) 

*  SwissPeaks360_DUO_2023 (14, 9) 

*  SwissPeaks360_DUO_2022 (30, 9) 

*  SwissPeaks360_DUO_2021 (14, 10) 

*  SwissPeaks360_DUO_2020 (18, 9) 



C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'] = df['Name'].str.replace(r"\s+", " ")
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\4092482852.py:40: FutureWarning: The default value of regex will change from True to False in a fu

In [7]:
for race in races:
    for year in years:
        try:
            SP_year = SP_itra_df[f'{race}_{year}']
            # Count the occurrences of each status
            status_counts = SP_year['Status'].value_counts()
            print(race, year, '\n', status_counts, '\n', '*'*40)
        except:
            pass

SwissPeaks360 2024 
 Finished    133
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2023 
 Finished    147
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2022 
 Finished    127
DNF         101
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2021 
 Finished    182
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2020 
 Finished    142
DNF          76
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2019 
 Finished    146
DNF          94
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360 2018 
 Finished    193
DNF         107
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360_DUO 2024 
 Finished    18
Name: Status, dtype: int64 
 ****************************************
SwissPeaks360_DUO 2023 
 Finished    14
Name: Status, dtype: int64 
 ********

In [8]:
SP_itra_df

{'SwissPeaks360_2024':                              Name  Age Sex ITRA_Nationality     Performance  \
 0                    Collison Kim   44   M              GBR 3 days 15:09:32   
 1            Bourguignon Matthieu   45   M              FRA 4 days 02:00:06   
 2                  Barbier Adrien   35   M              FRA 4 days 04:20:57   
 3    Galve Sainz De Varanda Jorge   39   M              ESP 4 days 04:20:57   
 4                   Orradre Inigo   46   M              ESP 4 days 07:09:31   
 ..                            ...  ...  ..              ...             ...   
 128             Hopkins Elizabeth   53   F              GBR 6 days 10:24:05   
 129                 Husson Gaetan   51   M              FRA 6 days 11:20:03   
 130                      Heng Bou   55   M              FRA 6 days 11:28:13   
 131                Das Neves Tony   52   M              FRA 6 days 11:39:30   
 132               Schmitt Vincent   57   M              FRA 6 days 12:04:14   
 
      Performanc

In [9]:
# Append using pd.concat
SP_itra_df = pd.concat(SP_itra_df)
SP_itra_df = SP_itra_df.reset_index(drop = True)
SP_itra_df.isna().sum()

Name                      0
Age                       0
Sex                       7
ITRA_Nationality         14
Performance             398
Performance_Seconds     398
Status                    0
Year                      0
Race                      0
Race Score             1528
dtype: int64

In [10]:
SP_itra_df

,Name,Age,Sex,ITRA_Nationality,Performance,Performance_Seconds,Status,Year,Race,Race Score
0,Collison Kim,44,M,GBR,3 days 15:09:32,313772.0,Finished,2024,SwissPeaks360,NaN
1,Bourguignon Matthieu,45,M,FRA,4 days 02:00:06,352806.0,Finished,2024,SwissPeaks360,NaN
2,Barbier Adrien,35,M,FRA,4 days 04:20:57,361257.0,Finished,2024,SwissPeaks360,NaN
3,Galve Sainz De Varanda Jorge,39,M,ESP,4 days 04:20:57,361257.0,Finished,2024,SwissPeaks360,NaN
4,Orradre Inigo,46,M,ESP,4 days 07:09:31,371371.0,Finished,2024,SwissPeaks360,NaN
...,...,...,...,...,...,...,...,...,...,...
1537,Binder Hubert,52,M,FRA,NaT,NaN,DNF,2020,SwissPeaks360_DUO,NaN
1538,Binder Marie,50,F,FRA,NaT,NaN,DNF,2020,SwissPeaks360_DUO,NaN
1539,Braga Andrea,32,M,FRA,NaT,NaN,DNF,2020,SwissPeaks360_DUO,NaN
1540,Krumm Tobias,43,M,GER,NaT,NaN,DNF,2020,SwissPeaks360_DUO,NaN


In [11]:
def categorize_age(age):
    if age < 40: # under40
        return 'SEN'
    elif age < 50: #40-49
        return 'V1'
    elif age < 60: #50-59
        return 'V2'
    elif age < 70: #60-69
        return 'V3'
    elif age >= 70:
        return 'V4' # Over 70
    else:
        return '-'
    
SP_itra_df['Age'] = SP_itra_df['Age'].str.replace('-', '0')
SP_itra_df['Age'] = pd.to_numeric(SP_itra_df['Age'], errors='coerce')

SP_itra_df['Age Category'] = SP_itra_df['Age'].apply(categorize_age)


## EDA of ITRA 

In [12]:
SP_itra_df[SP_itra_df['Sex'].isna()]

,Name,Age,Sex,ITRA_Nationality,Performance,Performance_Seconds,Status,Year,Race,Race Score,Age Category
106,Ravat Olivier,NaN,NaN,CAN,6 days 02:39:29,527969.0,Finished,2024,SwissPeaks360,NaN,-
242,Grondin Eric,NaN,NaN,FRA,5 days 22:30:31,513031.0,Finished,2023,SwissPeaks360,NaN,-
339,Bryant Shane,44.0,NaN,USA,5 days 12:26:00,476760.0,Finished,2022,SwissPeaks360,NaN,V1
363,Jones David,60.0,NaN,GBR,5 days 22:14:05,512045.0,Finished,2022,SwissPeaks360,NaN,V3
444,Fleith Michel,44.0,NaN,FRA,NaT,NaN,DNF,2022,SwissPeaks360,NaN,V1
552,Carlier Julien,NaN,NaN,FRA,5 days 02:48:27,442107.0,Finished,2021,SwissPeaks360,NaN,-
782,Milewicz Daniel,NaN,NaN,POL,4 days 17:51:24,409884.0,Finished,2020,SwissPeaks360,NaN,-


In [13]:
SP_itra_df['Sex'] = SP_itra_df['Sex'].fillna('M')


### Extracting Finish Category

In [21]:
# Convert to timedelta and get total hours (handling NaT)
SP_itra_df['Performance_Hours'] = pd.to_timedelta(
    SP_itra_df['Performance'], errors='coerce').dt.total_seconds() / 3600  # Convert seconds to hours

# Create a new column 'Finish Category'
def categorize_duration(hours):
    if pd.isna(hours):
        return 'DNF'  # Handle NaT values
    elif hours < 50:
        return '40-49 hours'
    elif hours < 150:
        num = int(hours // 10) * 10
        num1 = num +9
        return f'{num}-{num1} hours'  # Round to nearest 10 up to 150
    elif hours <= 156:
        num = int(hours // 10) * 10
        num1 = num +6
        return f'{num}-{num1} hours'  # Round to nearest 10 up to 150
    else:
        return 'Over-156 hours'
    
# Apply categorization with proper indexing
SP_itra_df['Finish Category'] = SP_itra_df['Performance_hours'].apply(categorize_duration)


# Define the desired order of categories
finish_category_order = [
    '40-49 hours',
    '50-59 hours',
    '60-69 hours',
    '70-79 hours',
    '80-89 hours', 
    '90-99 hours',
    '100-109 hours',
    '110-119 hours',
    '120-129 hours', 
    '130-139 hours', 
    '140-149 hours',
    'Over-150 hours', 
    'DNF'
]

# Set 'Finish Category' as a categorical column with the defined order
SP_itra_df['Finish Category'] = pd.Categorical(
    SP_itra_df['Finish Category'],
    categories = finish_category_order,
    ordered = True
)


# Display the DataFrame
SP_itra_df[['Year','Race', 'Performance_hours', 'Finish Category']]

    

C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\2527485093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP_itra_df['Performance_Hours'] = pd.to_timedelta(
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\2527485093.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP_itra_df['Finish Category'] = SP_itra_df['Performance_hours'].apply(categorize_duration)
C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\2527485093.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,Year,Race,Performance_hours,Finish Category
0,2024,SwissPeaks360,87.158889,80-89 hours
1,2024,SwissPeaks360,98.001667,90-99 hours
2,2024,SwissPeaks360,100.349167,100-109 hours
3,2024,SwissPeaks360,100.349167,100-109 hours
4,2024,SwissPeaks360,103.158611,100-109 hours
...,...,...,...,...
1537,2020,SwissPeaks360_DUO,NaN,DNF
1538,2020,SwissPeaks360_DUO,NaN,DNF
1539,2020,SwissPeaks360_DUO,NaN,DNF
1540,2020,SwissPeaks360_DUO,NaN,DNF


In [22]:
# grabbing all finishers 
SP_itra_df['Performance'] = SP_itra_df['Performance'].astype('str')

SP_itra_df = SP_itra_df[[
    'Name', 'ITRA_Nationality', 'Sex',  'Age', 'Age Category',
    'Performance','Performance_Seconds', 'Performance_Hours',
    'Status', 'Finish Category',
    'Race', 'Year',
]]
SP_itra_df
# SP_itra_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_itra_including_DNF.xlsx' , index = False)



C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\162484405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP_itra_df['Performance'] = SP_itra_df['Performance'].astype('str')


,Name,ITRA_Nationality,Sex,Age,Age Category,Performance,Performance_Seconds,Performance_Hours,Status,Finish Category,Race,Year
0,Collison Kim,GBR,M,NaN,-,3 days 15:09:32,313772.0,87.158889,Finished,80-89 hours,SwissPeaks360,2024
1,Bourguignon Matthieu,FRA,M,NaN,-,4 days 02:00:06,352806.0,98.001667,Finished,90-99 hours,SwissPeaks360,2024
2,Barbier Adrien,FRA,M,NaN,-,4 days 04:20:57,361257.0,100.349167,Finished,100-109 hours,SwissPeaks360,2024
3,Galve Sainz De Varanda Jorge,ESP,M,NaN,-,4 days 04:20:57,361257.0,100.349167,Finished,100-109 hours,SwissPeaks360,2024
4,Orradre Inigo,ESP,M,NaN,-,4 days 07:09:31,371371.0,103.158611,Finished,100-109 hours,SwissPeaks360,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,Binder Hubert,FRA,M,NaN,-,NaT,NaN,NaN,DNF,DNF,SwissPeaks360_DUO,2020
1538,Binder Marie,FRA,F,NaN,-,NaT,NaN,NaN,DNF,DNF,SwissPeaks360_DUO,2020
1539,Braga Andrea,FRA,M,NaN,-,NaT,NaN,NaN,DNF,DNF,SwissPeaks360_DUO,2020
1540,Krumm Tobias,GER,M,NaN,-,NaT,NaN,NaN,DNF,DNF,SwissPeaks360_DUO,2020


In [23]:
SP_itra_df[(SP_itra_df['Year']=='2018')|
            (SP_itra_df['Year']=='2019')|
            (SP_itra_df['Year']=='2021')|
            (SP_itra_df['Year']=='2022')|
            (SP_itra_df['Year']=='2023')|
            (SP_itra_df['Year']=='2024')].groupby(['Status', 'Year'])['Status' ].count().to_frame('count').reset_index()

,Status,Year,count
0,DNF,2018,107
1,DNF,2019,94
2,DNF,2022,115
3,Finished,2018,193
4,Finished,2019,146
5,Finished,2021,196
6,Finished,2022,143
7,Finished,2023,161
8,Finished,2024,151


### Extracting only Finishers

In [25]:
# SP_itra_df
SP_itra_no_DNF = SP_itra_df[(SP_itra_df['Status'] != 'DNF') ].reset_index(drop = True)
SP_itra_no_DNF = SP_itra_no_DNF[[
    'Name', 'ITRA_Nationality', 'Sex',  'Age', 'Age Category',
    'Performance','Performance_Seconds','Status',
    'Race', 'Year'
]]

In [26]:
SP_itra_no_DNF['Status'].unique()

array(['Finished'], dtype=object)

In [27]:
SP_itra_no_DNF.isna().sum()

Name                     0
ITRA_Nationality        14
Sex                      0
Age                    824
Age Category             0
Performance              0
Performance_Seconds      0
Status                   0
Race                     0
Year                     0
dtype: int64

In [28]:
SP_itra_no_DNF.head()

,Name,ITRA_Nationality,Sex,Age,Age Category,Performance,Performance_Seconds,Status,Race,Year
0,Collison Kim,GBR,M,NaN,-,3 days 15:09:32,313772.0,Finished,SwissPeaks360,2024
1,Bourguignon Matthieu,FRA,M,NaN,-,4 days 02:00:06,352806.0,Finished,SwissPeaks360,2024
2,Barbier Adrien,FRA,M,NaN,-,4 days 04:20:57,361257.0,Finished,SwissPeaks360,2024
3,Galve Sainz De Varanda Jorge,ESP,M,NaN,-,4 days 04:20:57,361257.0,Finished,SwissPeaks360,2024
4,Orradre Inigo,ESP,M,NaN,-,4 days 07:09:31,371371.0,Finished,SwissPeaks360,2024


In [29]:
SP_itra_no_DNF.groupby(['Status', 'Year'])['Status', 'Year'].count()

C:\Users\Karina\AppData\Local\Temp\ipykernel_3192\74827669.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  SP_itra_no_DNF.groupby(['Status', 'Year'])['Status', 'Year'].count()


Status  Year
Status   Year              
Finished 2018     193   193
         2019     146   146
         2020     154   154
         2021     196   196
         2022     143   143
         2023     161   161
         2024     151   151

### This data might be used later data Visualisations

In [31]:
# grabbing all finishers 
SP_itra_no_DNF['Performance'] = SP_itra_no_DNF['Performance'].astype('str')

SP_itra_no_DNF.to_excel(f'Data/SwissPeaks360_itra_no_DNF.xlsx' , index = False)